In [15]:
import requests
from bs4 import BeautifulSoup
import re

In [1]:
!pip install fake_useragent


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 11 news categories
#news_links =['aipl', 'ait', 'aspt', 'asc', 'aie', 'amov','ahel','aopl','asoc','acul','acn']
#news_categories=['政治','科技','運動','證卷','產經','娛樂','生活','國際','社會','文化','兩岸']
news_links =['','entertainment','world','politics','society','finance','sports','fun', 'style', 'game-3c','topic/international-womens-day']
news_categories=['焦點','娛樂影劇','國際','政治','社會地方','財經','運動','玩樂','品味','遊戲3C']
base_url = 'https://tw.news.yahoo.com/'

In [3]:
import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime, timedelta
from fake_useragent import UserAgent

In [4]:
user_agent = UserAgent()

In [5]:
%%time
# 存放資料之變數
links = []
titles = []
dates = []
contents = []
categories = []
item_id = []
photo_links = []
ts=[]
# 這裡我們需要用for迴圈爬取11個類別
# Here we need to crawl 11 categories using for loop
for i, url_short_name in enumerate(news_links):  # 針對每一類 共有11類

    category = news_categories[i]  # '政治' #類別名稱紀錄起來 

    # Categorical url link
    category_url = base_url + url_short_name 
    print("Getting categorical news:", category)

    # Request the categorical news page
    req = requests.get(category_url, headers={'user-agent': user_agent.random}, timeout=5)
    page = BeautifulSoup(req.text, 'lxml')
    
    # 抓新聞列表(待改進)
    items = page.findAll('a', attrs={'class': ['js-content-viewer', 'rapidnofollow', 'D(ib)']})

    # 開始爬取這個類別的新聞
    serial_no = 1
    news_no=1
    for item_j, item in enumerate(items, start=1):  # 針對每一篇新聞抓取細節
        title = item.get_text(strip=True) or "No title"
        link = item.get('href')  # 修正 `item.find('a').get('href')` 錯誤
        print(news_no, '--', title)
        news_no += 1
        # img
        full_link = base_url + link if link.startswith('/') else link  # 確保是完整網址

       
        try:
            news_req = requests.get(full_link, headers={'user-agent': user_agent.random}, timeout=5)
            news_page = BeautifulSoup(news_req.text, 'lxml')

            img_tag = news_page.find('div', {'class': "caas-img-container"})
            photo_link = img_tag.find('img')['data-src']
            print("Image URL:", photo_link)

            news_date = news_page.find('time', {'class': "caas-attr-meta-time"}).text.strip()
            news_date = news_date.replace("上午", "AM").replace("下午", "PM")
            news_date = re.sub(r'(\d+)月', lambda x: f"{int(x.group(1)):02}月", news_date)
            dt = datetime.strptime(news_date, "%Y年%m月%d日 %p%I:%M")
            formatted_date = dt.strftime("%Y-%m-%d")
            print(formatted_date)
            
            categories.append(category)
            titles.append(title)
            photo_links.append(photo_link)
            dates.append(formatted_date)

            dtstr = dt.strftime("%Y%m%d")
            item_id.append(url_short_name + "_" + dtstr + "_" + str(serial_no))

            
            serial_no += 1
            try:
                content_paragraphs = news_page.findAll('p')
                content_text = "\n".join([p.get_text(strip=True) for p in content_paragraphs if p.get_text(strip=True)])
                cont = content_text
                cont = re.sub(r'（.*?）|\(圖／.*?\)|文／.*?整理報導', '', cont)
                cont = re.sub('\n','', cont) 
                cont = re.sub('（譯者：.*','', cont) 
                contents.append(cont)
            except:
                content_text = "No content found"
            print(content_paragraphs)
            
        except Exception as e:
            print("Error fetching :", e)

        #抓新聞內容
        if not link.startswith(base_url):
            link = base_url+link
            print(link)
        else:
            print(link)
        links.append(link)

        

        # Find content
       
        # Remove unnecessary content
        try:
            content.find('div', {'class':"gmailNews"}).decompose()
        except:
            pass
        # 額外插入的延伸閱讀區塊-->移除之
        try:
            content.find('div', {'class':"paragraph moreArticle"}).decompose()
        except:
            pass
        # 額外插入的訂閱連結-->移除之
        try:
            content.find('div', class_="SubscriptionInner mySubscriptionInner").decompose()
        except:
            pass

        # 額外插入的圖片與文字說明區塊-->移除之
        try:
            for media in content.findAll('div', {'class':"media"}):
                media.decompose()
        except:
            pass

        # Remove unnecessary content
        # cont = content.text
        # cont = re.sub('（圖+?）','', cont)
        # cont = re.sub('（編輯：.*','', cont) 
        # cont = re.sub('（譯者：.*','', cont) 
        
        # contents.append(cont)
 

        # if item_j >= 10: # Here we crawl only 4 pieces of news for each category, in order to save time.
        #     break 
        
    #     time.sleep(10)  # 遵守爬蟲禮節，請小睡一下


Getting categorical news: 焦點
1 -- 台股重挫308點 失守22000關卡
Image URL: https://s.yimg.com/ny/api/res/1.2/3qEveVGKp0705iucq1UPTw--/YXBwaWQ9aGlnaGxhbmRlcjt3PTY0MDtoPTQyNw--/https://s.yimg.com/os/creatr-uploaded-images/2024-10/83267440-8445-11ef-b675-1135712e37b5
2025-03-27
[<p>美國總統川普準備徵收汽車關稅，美股主要指數全數下跌，台積電重挫4.09%，台積電台北現股今（27）日早盤以961元開出後失守960價位，終場收在958元，下跌22元或2.24%；台股以22093點開出後失守22000關卡，收盤以21951點作收，下跌308點。</p>, <p>川普已經正式宣布，將對所有不在美國製造的汽車徵收25%的進口稅，美股四大指數下跌，費城半導體指數下滑3.27%，台積電ADR收盤大跌4.09%，報每股173.50美元，換算並折合台幣後是每股1,148.15元，較台北交易股票溢價率為17.16%。</p>, <p>台股今早盤以22093點開出後，盤中失守22000點關卡，最低觸及21919點，收盤以21951點作收，下跌308點；台積電以961元開出，盤中最低觸及958元，終場以958元作收，下跌22元或2.24％。</p>, <p>凱基投顧報告指出，周三大盤高點持續逼近月線壓力，但買盤仍無力追價，成交量急凍至近年新低水準，導致再度挑戰月線未果，不過，盤面上漲家數約占三分之二，有別前一日下跌家數約占七成，顯示盤面籌碼趨於穩定，而且盤中拉回尚能力守短期均線，技術面呈現價穩量縮走勢，有利短線持續挑戰月線，站上月線後就有機會進一步往上挑戰3月13日長黑高點22552點，站上22552點後就能完成底部型態，使反彈走勢進一步延伸。</p>, <p>凱基投顧表示，目前盤面類股輪動快速，因此，操作上仍不宜過度追價，但大盤底部型態持續醞釀，部分個股開始率先表態強攻，選股可優先聚焦具利多題材以及股價轉強收復均線壓力之強勢個股。</p>]
https://tw.news.yahoo.com/https://tw.stock.yahoo.

IndexError: list index out of range

In [6]:
titles

['台股重挫308點 失守22000關卡',
 '賴士葆撞人畫面外流 警被起訴',
 '墾丁爆非法民宿飆價 署長說重話',
 '我軍艦與陸漁船擦撞 海軍發聲',
 '受陸官媒訪問批賴 區桂芝被檢舉',
 '台灣首例 諸慶恩判刑後病故翻案',
 '身價達2.1兆元 大陸首富換人當',
 '75人聲明⋯言論自由尺度掀論',
 '川普：對非美製汽車徵收25%關稅',
 '管制武統言論？法務部：有學者建議訂專法',
 '稱「亞亞」是極少數個案 羅文嘉籲勿產生歧視',
 '評論／「亞亞案」民進黨100％言論自由去哪？',
 '台灣街頭驚見路邊堆600萬現金 路人報警',
 '上影印店「單次就破萬」網揭賺錢之道',
 '「學聰明了」提早掃墓避塞車 釣出一堆同類',
 '長輩退休超愛去對岸旅遊？網揭關鍵',
 '買完上下舖超後悔⋯抱怨睡眠品質變差 過來人提解方',
 '腫瘤仍增長 陳文茜曝免疫治療失敗',
 '秀秀子曬雙證照 揭昔日游泳教練身分',
 '陳孝萱帥兒變學霸 19歲模樣曝光',
 '被抓包販賣粉絲禮物 曾黎公司澄清',
 '家寧一家涉逃漏稅 Andy現身調查局',
 '北京正在主導國際經貿體系！前貿易代表坦承：美國已生活在中國建構的世界裡',
 '搭飛機乘客躲廁所「偷偷1舉動」\u3000機長廣播怒斥：會害死188人',
 '副總統遭逮捕內亂不斷\u3000聯合國預警南蘇丹恐有大規模衝突',
 '4姊姊共同照顧「植物人弟」14年！竟被姪女告上法院\u3000要求獨自繼承遺產',
 '男子不停打嗝「持續2年多」！累到住院\u3000血液檢查意外揪出病因',
 '中國台辦設舉報專欄追責「台獨打手」\u3000民眾黨批：無助兩岸和平',
 '吳典蓉專欄：亞亞如何成為中共設下的圈套？',
 '工商社論》丞相起風了？賴政府不該再對核電欲迎還拒',
 '學者觀點－借鏡國際趨勢 跨國的海底電纜互聯 增加能源韌性',
 '鎖定八炯、閩南狼？\u3000國台辦設「台獨舉報專欄」',
 'No title',
 'No title',
 '黃子佼判決書曝光！苗博雅重批：不該繼續在社會上當意見領袖',
 '信評機構警告美國財政惡化\u3000利息支出占收入的比率將從9％升至30％',
 '保健食品關稅30%\u3000財長：部分分年逐步調降',
 '台南連鎖牛排店用餐⋯5歲童：怎麼硬硬的！

In [7]:
contents

['美國總統川普準備徵收汽車關稅，美股主要指數全數下跌，台積電重挫4.09%，台積電台北現股今日早盤以961元開出後失守960價位，終場收在958元，下跌22元或2.24%；台股以22093點開出後失守22000關卡，收盤以21951點作收，下跌308點。川普已經正式宣布，將對所有不在美國製造的汽車徵收25%的進口稅，美股四大指數下跌，費城半導體指數下滑3.27%，台積電ADR收盤大跌4.09%，報每股173.50美元，換算並折合台幣後是每股1,148.15元，較台北交易股票溢價率為17.16%。台股今早盤以22093點開出後，盤中失守22000點關卡，最低觸及21919點，收盤以21951點作收，下跌308點；台積電以961元開出，盤中最低觸及958元，終場以958元作收，下跌22元或2.24％。凱基投顧報告指出，周三大盤高點持續逼近月線壓力，但買盤仍無力追價，成交量急凍至近年新低水準，導致再度挑戰月線未果，不過，盤面上漲家數約占三分之二，有別前一日下跌家數約占七成，顯示盤面籌碼趨於穩定，而且盤中拉回尚能力守短期均線，技術面呈現價穩量縮走勢，有利短線持續挑戰月線，站上月線後就有機會進一步往上挑戰3月13日長黑高點22552點，站上22552點後就能完成底部型態，使反彈走勢進一步延伸。凱基投顧表示，目前盤面類股輪動快速，因此，操作上仍不宜過度追價，但大盤底部型態持續醞釀，部分個股開始率先表態強攻，選股可優先聚焦具利多題材以及股價轉強收復均線壓力之強勢個股。',
 '去年7月間，國民黨立委賴士葆駕車行駛在台北市大安區時，不慎撞傷走在斑馬線上2位女性行人，沒想到事後現場車禍照片及車內行車紀錄畫面卻被媒體刊登，2名女子事後與賴士葆和解，卻無法忍受車禍照片被刊出，因此提出告訴，大安分局最後查出是交通分隊洪姓男警員涉有重嫌，將他函送台北地檢署偵辦，檢察官調查後認為洪姓警員涉犯《個人資料保護法》及《刑法》洩密罪，依法起訴。賴士葆於去年7月9日晚間9時許，駕駛一輛白色休旅車行經台北市瑞安街轉彎處，撞飛走在斑馬線上2位女性行人，造成2名行人分別骨折、頭部撕裂傷送醫。事後賴士葆出面鞠躬道歉，表示因天色昏暗及A柱死角才釀禍。案發當下，大安分局瑞安街派出所洪姓女警員獲報後前往現場處理，當她抵達現場後就利用M-Police查詢汽車駕駛人賴士葆、被撞傷的蔡姓、張姓女子的個人資料，並用手機拍照

In [8]:
item_id

['_20250327_1',
 '_20250327_2',
 '_20250327_3',
 '_20250327_4',
 '_20250327_5',
 '_20250327_6',
 '_20250327_7',
 '_20250327_8',
 '_20250327_9',
 '_20250327_10',
 '_20250327_11',
 '_20250327_12',
 '_20250327_13',
 '_20250326_14',
 '_20250326_15',
 '_20250326_16',
 '_20250326_17',
 '_20250327_18',
 '_20250327_19',
 '_20250327_20',
 '_20250327_21',
 '_20250327_22',
 '_20250327_23',
 '_20250327_24',
 '_20250327_25',
 '_20250327_26',
 '_20250327_27',
 '_20250327_28',
 '_20250327_29',
 '_20250327_30',
 '_20250327_31',
 '_20250326_32',
 '_20250327_33',
 '_20250327_34',
 '_20250327_35',
 '_20250327_36',
 '_20250327_37',
 '_20250327_38',
 '_20250327_39',
 '_20250327_40',
 '_20250327_41',
 '_20250327_42',
 '_20250327_43',
 '_20250327_44',
 '_20250327_45',
 '_20250326_46',
 '_20250327_47',
 '_20250327_48',
 '_20250327_49',
 '_20250327_50',
 '_20250327_51',
 '_20250327_52',
 '_20250327_53',
 '_20250327_54',
 'entertainment_20250327_1',
 'entertainment_20250327_2',
 'entertainment_20250327_3',
 'en

In [9]:
data = zip(item_id, dates, categories, titles, contents, links, photo_links)
df = pd.DataFrame(list(data), columns=['item_id','date','category','title','content','link','photo_link'])

In [10]:
df.head(2)

,item_id,date,category,title,content,link,photo_link
0,_20250327_1,2025-03-27,焦點,台股重挫308點 失守22000關卡,美國總統川普準備徵收汽車關稅，美股主要指數全數下跌，台積電重挫4.09%，台積電台北現股今日...,https://tw.news.yahoo.com/https://tw.stock.yah...,https://s.yimg.com/ny/api/res/1.2/3qEveVGKp070...
1,_20250327_2,2025-03-27,焦點,賴士葆撞人畫面外流 警被起訴,去年7月間，國民黨立委賴士葆駕車行駛在台北市大安區時，不慎撞傷走在斑馬線上2位女性行人，沒想...,https://tw.news.yahoo.com/%E8%B3%B4%E5%A3%AB%E...,https://s.yimg.com/ny/api/res/1.2/PPzJSVLdFBTm...


In [11]:
df.shape

(216, 7)

In [12]:
df.content[0]

'美國總統川普準備徵收汽車關稅，美股主要指數全數下跌，台積電重挫4.09%，台積電台北現股今日早盤以961元開出後失守960價位，終場收在958元，下跌22元或2.24%；台股以22093點開出後失守22000關卡，收盤以21951點作收，下跌308點。川普已經正式宣布，將對所有不在美國製造的汽車徵收25%的進口稅，美股四大指數下跌，費城半導體指數下滑3.27%，台積電ADR收盤大跌4.09%，報每股173.50美元，換算並折合台幣後是每股1,148.15元，較台北交易股票溢價率為17.16%。台股今早盤以22093點開出後，盤中失守22000點關卡，最低觸及21919點，收盤以21951點作收，下跌308點；台積電以961元開出，盤中最低觸及958元，終場以958元作收，下跌22元或2.24％。凱基投顧報告指出，周三大盤高點持續逼近月線壓力，但買盤仍無力追價，成交量急凍至近年新低水準，導致再度挑戰月線未果，不過，盤面上漲家數約占三分之二，有別前一日下跌家數約占七成，顯示盤面籌碼趨於穩定，而且盤中拉回尚能力守短期均線，技術面呈現價穩量縮走勢，有利短線持續挑戰月線，站上月線後就有機會進一步往上挑戰3月13日長黑高點22552點，站上22552點後就能完成底部型態，使反彈走勢進一步延伸。凱基投顧表示，目前盤面類股輪動快速，因此，操作上仍不宜過度追價，但大盤底部型態持續醞釀，部分個股開始率先表態強攻，選股可優先聚焦具利多題材以及股價轉強收復均線壓力之強勢個股。'

In [13]:
df.to_csv("yahoo_category_news.csv", sep="|", index=False)

In [14]:
photo_links

['https://s.yimg.com/ny/api/res/1.2/3qEveVGKp0705iucq1UPTw--/YXBwaWQ9aGlnaGxhbmRlcjt3PTY0MDtoPTQyNw--/https://s.yimg.com/os/creatr-uploaded-images/2024-10/83267440-8445-11ef-b675-1135712e37b5',
 'https://s.yimg.com/ny/api/res/1.2/PPzJSVLdFBTm6FiLGSbbxA--/YXBwaWQ9aGlnaGxhbmRlcjt3PTk2MDtoPTU0MA--/https://media.zenfs.com/ko/taisounds_com_380/463fd7df7b76b22d0fa13d9593fb3bff',
 'https://s.yimg.com/ny/api/res/1.2/E9TwmMfRm0bQbJPeG0cHTA--/YXBwaWQ9aGlnaGxhbmRlcjt3PTY0MDtoPTQyNw--/https://media.zenfs.com/zh-tw/cna.com.tw/c747aa476efff1129af9af39904ac0ec',
 'https://s.yimg.com/ny/api/res/1.2/_4BjyXb7OV7amX_YUfvXvg--/YXBwaWQ9aGlnaGxhbmRlcjt3PTY0MDtoPTQyNg--/https://s.yimg.com/os/creatr-uploaded-images/2025-03/1a99aa20-0ab2-11f0-91fd-9765ec42edcc',
 'https://s.yimg.com/ny/api/res/1.2/_FP1JJNJzanEfvomS9wVQg--/YXBwaWQ9aGlnaGxhbmRlcjt3PTY0MDtoPTQ1NA--/https://media.zenfs.com/ko/cna.com.tw/809508f49da456a23a6ded63cfd2a590',
 'https://s.yimg.com/ny/api/res/1.2/fw.sj1.tLBM79qJKNKxnWQ--/YXBwaWQ9aGlnaGxh